## Quality assurance when you have fully labelled data

In this example, our data contains a fully-populated ground-truth column called `cluster` that enables us to perform accuracy analysis of the final model


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/duckdb/accuracy_analysis_from_labels_column.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [2]:
from splink.datasets import splink_datasets
import altair as alt

df = splink_datasets.fake_1000

df.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
0,0,Robert,Alan,1971-06-24,NaN,robert255@smith.net,0
1,1,Robert,Allen,1971-05-24,NaN,roberta25@smith.net,0


In [3]:
from splink.linker import Linker


from splink.blocking_rule_library import block_on
import splink.comparison_template_library as ctl
import splink.comparison_library as cl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "comparisons": [
        ctl.NameComparison("first_name"),
        ctl.NameComparison("surname"),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["month", "month", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

In [4]:
from splink.database_api import DuckDBAPI

db_api = DuckDBAPI()
linker = Linker(df, settings, database_api=db_api)
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.00333.
This means that amongst all possible pairwise record comparisons, one in 300.13 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 1,664.29 matching pairs


In [5]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=5)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [6]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.417 in the m_probability of surname, level `Exact match on surname`
Iteration 2: Largest change in params was 0.121 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0354 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0127 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00539 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.0025 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.0012 in probability_two_random_records_match
Iteration 8: Largest change in 

In [7]:
linker.truth_space_table_from_labels_column(
    "cluster", match_weight_round_to_nearest=0.1
).as_pandas_dataframe(limit=5)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'dob':
    m values not fully trained


,truth_threshold,match_probability,row_count,p,n,tp,tn,fp,fn,P_rate,...,precision,recall,specificity,npv,accuracy,f1,f2,f0_5,p4,phi
0,-25.2,2.594442e-08,4353.0,2031.0,2322.0,2031.0,0.0,2322.0,0.0,0.466575,...,0.466575,1.000000,0.000000,1.000000,0.466575,0.636278,0.813898,0.522296,0.000000,0.000000
1,-22.9,1.277653e-07,4353.0,2031.0,2322.0,2030.0,0.0,2322.0,1.0,0.466575,...,0.466452,0.999508,0.000000,0.000000,0.466345,0.636065,0.813562,0.522146,0.000000,-0.016208
2,-16.5,1.078947e-05,4353.0,2031.0,2322.0,2028.0,0.0,2322.0,3.0,0.466575,...,0.466207,0.998523,0.000000,0.000000,0.465886,0.635637,0.812891,0.521847,0.000000,-0.028080
3,-15.2,2.656638e-05,4353.0,2031.0,2322.0,2027.0,611.0,1711.0,4.0,0.466575,...,0.542269,0.998031,0.263135,0.993496,0.606019,0.702721,0.854409,0.596773,0.522674,0.374063
4,-14.3,4.957349e-05,4353.0,2031.0,2322.0,2019.0,977.0,1345.0,12.0,0.466575,...,0.600178,0.994092,0.420758,0.987867,0.688261,0.748471,0.878743,0.651837,0.659950,0.493913


In [8]:
linker.roc_chart_from_labels_column("cluster")
# Can also do linker.precision_recall_chart_from_labels_column("cluster")


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'dob':
    m values not fully trained


alt.Chart(...)

In [9]:
linker.confusion_matrix_from_labels_column("cluster")


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'dob':
    m values not fully trained


alt.HConcatChart(...)

In [10]:
# Plot some false positives
linker.prediction_errors_from_labels_column(
    "cluster", include_false_negatives=True, include_false_positives=True
).as_pandas_dataframe(limit=5)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'dob':
    m values not fully trained


,clerical_match_score,found_by_blocking_rules,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,...,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,cluster_l,cluster_r,match_key
0,1.0,False,-25.166033,2.656252e-08,417,418,Florence,Brown,0,0.208768,...,0.00123,0.426409,1.0,fb@reose.cem,f@b@reese.com,0,0.001023,108,108,2
1,1.0,False,-22.906005,1.272346e-07,796,797,Taylor,None,-1,1.000000,...,0.00738,0.426409,1.0,jt40o@combs.net,jt40@cotbs.nm,0,0.001023,201,201,2
2,1.0,False,-22.906005,1.272346e-07,452,454,None,Davies,-1,1.000000,...,0.01599,0.426409,1.0,rd@lewis.com,idlewrs.cocm,0,0.001023,115,115,2
3,1.0,True,-16.474084,1.098504e-05,594,595,Grace,Grace,3,86.337813,...,0.00123,0.426409,1.0,gk@frey-robinson.org,rgk@frey-robinon.org,0,0.001023,146,146,0
4,1.0,False,-15.233498,2.595667e-05,150,151,Alfie,Kelly,0,0.208768,...,0.04920,0.426409,1.0,alfiekelly@walters.com,None,-1,1.000000,40,40,2


In [11]:
records = linker.prediction_errors_from_labels_column(
    "cluster", include_false_negatives=True, include_false_positives=True
).as_record_dict(limit=5)

linker.waterfall_chart(records)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'dob':
    m values not fully trained


alt.LayerChart(...)